In [9]:
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   ------------ --------------------------- 3.7/11.6 MB 21.7 MB/s eta 0:00:01
   -------------- ------------------------- 4.2/11.6 MB 9.3 MB/s eta 0:00:01
   ------------------- -------------------- 5.8/11.6 MB 9.3 MB/s eta 0:00:01
   ---------------------- ----------------- 6.6/11.6 MB 7.5 MB/s eta 0:00:01
   -------------------------- ------------- 7.6/11.6 MB 7.1 MB/s eta 0:00:01
   ----------------------------- ---------- 8.7/11.6 MB 6.8 MB/s eta 0:00:01
   -------------------------------- ------- 9.4/11.6 MB 6.4 MB/s eta 0:00:01
   ------------------------------------- -- 10.7/11.6 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------  11.5/11.6 MB 6.0 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 5.8 MB/s eta 0:00:00


In [5]:
import os
import numpy as np
import json
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Load data
data_dir = './keypoints_output'
data = []
labels = []

for filename in os.listdir(data_dir):
    if filename.endswith('.json'):
        file_path = os.path.join(data_dir, filename)
        with open(file_path, 'r') as f:
            sequence = json.load(f)  # Load JSON data
        data.append(np.array(sequence))  # Convert to NumPy array
        labels.append(filename.split('_')[0])  # Assuming label is before first underscore

# Encode labels
le = LabelEncoder()
encoded_labels = le.fit_transform(labels)
y_encoded = to_categorical(encoded_labels)

# Pad sequences
max_frames = max([seq.shape[0] for seq in data])
data_padded = np.array([np.pad(seq, ((0, max_frames - seq.shape[0]), (0, 0)), mode='constant') for seq in data])

# Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data_padded, y_encoded, test_size=0.2, random_state=42)

# Build and compile model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking

model = Sequential()
model.add(Masking(mask_value=0., input_shape=(max_frames, data_padded.shape[2])))
model.add(LSTM(128, return_sequences=False))
model.add(Dense(y_encoded.shape[1], activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Save model
model.save('text_to_sign_model.h5')


IndexError: tuple index out of range